In [ ]:
import json
import pandas as pd
import C2Analyse
import C2Analyse as C2A
import matplotlib.pyplot as plt
import importlib
import re
import missingno as msno
import plotly.express as px
import numpy as np
from datetime import datetime
importlib.reload(C2A)
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

pd.options.display.max_columns = None
reload_data = 1